In [1]:
import numpy as np
import pandas as pd

# model_selection.splitter
from sklearn.model_selection import train_test_split   # **********************      
from sklearn.model_selection import GridSearchCV       # **********************
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_validate, cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


# metrics.scores
from sklearn.metrics import f1_score, roc_auc_score    # **********************
from sklearn.metrics import accuracy_score, precision_score, recall_score,  confusion_matrix

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# **************************************************

# Classifier ---------------------------------------
# from sklearn.linear_model import LogisticRegression # classifier
# from xgboost import XGBClassifier, XGBRFClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

# metrics.curves
# from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve
# from sklearn.metrics import roc_curve, plot_roc_curve
# metrics.reports
# from sklearn.metrics import classification_report, ConfusionMatrixDisplay
# --------------------------------------------------




<pre>
1) 타겟확인
2) 결측확인 : isna().sum()   fillna()
3) 숫자타입 : Object타입이 보이면 아래 작업 수행

#------------------ 글자 ==> 숫자 변환 ---------------
# Array(가로)     : LabelEncoding : 글자 --> 숫자    =========> fit(변환된결과)  df['CC']=변환된결과.reshape(-1,1)
# Array(가로)     : OneHotEncoding : 0~9 
# DataFraem(세로) : get_dummie

4) 학습 (train 8:2 val)

5) 테스트(test submit)  ---  답안지

## 분석 : 숫자, 결측

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("submission.csv")

In [3]:
train.shape, test.shape

((891, 12), (418, 11))

In [4]:
df = pd.concat([train, test], axis=0)

* 결측확인 - 숫자(평균)
<pre>
Age             263
Cabin          1014
Embarked          2

In [5]:
df.isna().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [6]:
# df[ (df['Pclass']==3) & (df['SibSp']==0) & (df['Parch']==0)]['Fare'].mean()  #9.09
df['Fare'].fillna(9.09, inplace=True)

In [7]:
#df['Age'].mean()
df['Age'].fillna(29.88, inplace=True)

In [8]:
df['Embarked'].fillna('S', inplace=True)

* Object 타입 - 최빈도
<pre>
 3   Name         1309 non-null   object   --- drop
 4   Sex          1309 non-null   object 
 8   Ticket       1309 non-null   object   --- drop
 10  Cabin        295 non-null    object   --- drop
 11  Embarked     1307 non-null   object   --- S

In [9]:
dict1 = {'male':0, 'female':1}
df['Sex'] = df['Sex'].map(dict1)

dict2 = {'S':0, 'C':1, 'Q':2}
df['Embarked'] = df['Embarked'].map(dict2)

* 불필요컬럼 삭제

In [10]:
# print(df.columns.tolist())
col = ['PassengerId',  'Name', 'Ticket',  'Cabin']
df = df.drop(col, axis=1)

In [11]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.0,3,0,22.0,1,0,7.2500,0
1,1.0,1,1,38.0,1,0,71.2833,1
2,1.0,3,1,26.0,0,0,7.9250,0
3,1.0,1,1,35.0,1,0,53.1000,0
4,0.0,3,0,35.0,0,0,8.0500,0


## 학습 테스트 나누기

In [12]:
train = df[~df['Survived'].isna()]   #891
test = df[df['Survived'].isna()]     #418

In [13]:
test.drop('Survived', axis=1, inplace=True)

f:\workspace\venv\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [14]:
train['Survived']

0      0.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
886    0.0
887    1.0
888    0.0
889    1.0
890    0.0
Name: Survived, Length: 891, dtype: float64

In [15]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,2
1,3,1,47.0,1,0,7.0000,0
2,2,0,62.0,0,0,9.6875,2
3,3,0,27.0,0,0,8.6625,0
4,3,1,22.0,1,1,12.2875,0


## 학습준비

In [16]:
y = train['Survived']
X = train.drop('Survived', axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [17]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((801, 7), (90, 7), (801,), (90,))

In [18]:
y_train

175    0.0
101    0.0
24     0.0
681    1.0
127    1.0
      ... 
236    0.0
621    1.0
130    0.0
605    0.0
54     0.0
Name: Survived, Length: 801, dtype: float64

In [19]:
y_val

692    1.0
393    1.0
122    0.0
1      1.0
865    1.0
      ... 
147    0.0
312    0.0
709    1.0
569    1.0
160    0.0
Name: Survived, Length: 90, dtype: float64

In [20]:
# model = RandomForestClassifier()
# model.fit(X_train, y_train)
# pred = model.predict(X_val)
# # f1_score(y_val, pred)

In [22]:
models = [DecisionTreeClassifier(), RandomForestClassifier()] #, GradientBoostingClassifier()]
for model in models:
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    f1 = f1_score(y_val, pred)
    acc = accuracy_score(y_val, pred)

In [23]:
f1, acc

(0.631578947368421, 0.7666666666666667)